In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn import linear_model, ensemble, tree, multioutput, preprocessing, model_selection, metrics
import xgboost as xgb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/compsectorprice.xlsx', index_col=0)
df['Sector'] = df['Sector'].fillna('Unknow')
df = df.fillna(0)

df['roa'] = (df['Net Income'] /  df['Total Assets'] )
df['roe'] = (df['Net Income'] /  df['Total Equity']) 
df['roc'] = (df['Operating Income (Loss)'] /  df['Total Assets'])
df['current_ratio'] = df['Total Current Assets'] / df['Total Current Liabilities']
df['quick_ratio'] = (df['Accounts & Notes Receivable'] + df['Cash, Cash Equivalents & Short Term Investments']) / df['Total Current Liabilities'] 
df['gross_margin'] = df['Gross Profit'] / df['Revenue']
df['net_income_margin'] = df['Net Income'] / df['Revenue']
fcf = df['Net Cash from Operating Activities'] - df['Net Cash from Financing Activities']
df['fcf'] = fcf
df['fcf_margin'] = fcf / df['Revenue']
owners_earnings = df['Net Income'] + df['Depreciation & Amortization'] + df['Change in Working Capital'] + df['Net Cash from Financing Activities']
df['owners_earnings'] = owners_earnings
df['owners_earnings_to_net_income'] = owners_earnings / df['Net Income']

In [ ]:
resumed_df = df[
    [
        'Ticker', 'Fiscal Year', 'Sector',
        'Revenue', 'Cost of Revenue', 'Net Income',
        'roa',
'roe',
'roc',
'current_ratio',
'quick_ratio',
'gross_margin',
'net_income_margin',
'fcf',
'fcf_margin',
'owners_earnings',
'owners_earnings_to_net_income',
    ]
].copy()

resumed_df['Revenue'] = resumed_df['Revenue'] /1000000
resumed_df['Cost of Revenue'] = resumed_df['Cost of Revenue'] /1000000
resumed_df['Net Income'] = resumed_df['Net Income'] /1000000
resumed_df['fcf'] = resumed_df['fcf'] /1000000
resumed_df['owners_earnings'] = resumed_df['owners_earnings'] /1000000

# resumed_df['Fiscal Year'] = pd.PeriodIndex(resumed_df['Fiscal Year'], freq='Y')
resumed_df = resumed_df.sort_values(by=['Ticker','Fiscal Year'], ascending=False)
resumed_df.loc[resumed_df['Ticker'] == resumed_df['Ticker'].shift(periods=1), 'nextYearRev'] = resumed_df['Revenue'].shift(periods=1)
resumed_df.loc[resumed_df['Ticker'] == resumed_df['Ticker'].shift(periods=1), 'nextYearCostRev'] = resumed_df['Cost of Revenue'].shift(periods=1)


In [ ]:
categorical_cols = resumed_df.select_dtypes('object').columns.tolist()
encoder = preprocessing.OneHotEncoder(sparse=False, handle_unknown='ignore').fit(resumed_df[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
resumed_df[encoded_cols] = encoder.transform(resumed_df[categorical_cols])
resumed_df = resumed_df.drop(columns=['Sector'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [ ]:
validation_data = resumed_df[resumed_df['nextYearRev'].isna()]

test_train_data = resumed_df.dropna()

In [ ]:
# X features
# y target
y = test_train_data.loc[:, test_train_data.columns.intersection(['nextYearRev','nextYearCostRev'])]
X = test_train_data.drop(columns=['Ticker', 'nextYearRev', 'nextYearCostRev'])

X.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
lin_reg = linear_model.LinearRegression()


In [ ]:
lin_reg.fit(X_train, y_train)


LinearRegression()

In [ ]:
lin_reg_predict = lin_reg.predict(X_test)


In [ ]:
print('lin_reg', lin_reg.score(X_test, y_test))


lin_reg -4372555173.579463


In [ ]:

lin_reg_scores = model_selection.cross_val_score(lin_reg, X, y, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (lin_reg_scores.mean(), lin_reg_scores.std()))


-483364578193.51 accuracy with a standard deviation of 593366480448.74
